# Installing Libraries

In [ ]:
!pip install pydantic-ai requests beautifulsoup4 gradio > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.43.0 which is incompatible.
google-auth-oauthlib 1.2.3 requires google-auth<2.42.0,>=2.15.0, but you have google-auth 2.43.0 which is incompatible.


# Setting Parameters

In [ ]:
# -------------------------------------------------------
# CONFIGURATION
# -------------------------------------------------------
from google.colab import userdata

MODEL_NAME = "gemini-2.5-flash-lite"
API_KEY = userdata.get('GeminiKey')

NEWS_URL = "https://techcrunch.com/feed/"


# LLM initialization

In [ ]:
# -------------------------------------------------------
# AGENT SETUP
# -------------------------------------------------------
from pydantic_ai import Agent
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider

provider = GoogleProvider(api_key=API_KEY)
model = GoogleModel(MODEL_NAME, provider=provider)

agent = Agent(model)
result = await agent.run("What is the meaning of life? Reply in one sentence")
print(result.output)

The meaning of life is a question that has been pondered by philosophers, theologians, and individuals throughout history, and there is no single, universally accepted answer.


# News scraping Tool

In [ ]:
# -------------------------------------------------------
# TOOL: scrape real news (Reuters RSS)
# -------------------------------------------------------
from pydantic_ai import Agent, Tool
from bs4 import BeautifulSoup
import requests

@Tool
def scrape():
    print("\n🟦 [TOOL CALLED] scrape() was triggered by the LLM")
    print("🔄 Step 1: Fetching news feed...")

    try:
        response = requests.get(NEWS_URL, timeout=10)
        soup = BeautifulSoup(response.text, "xml")

        items = soup.find_all("item")[:5]
        result = ""

        print("🔄 Step 2: Extracting top headlines...")
        for item in items:
            title = item.find("title").text
            link = item.find("link").text
            result += f"- {title}\n  {link}\n\n"

        print("\n✅ [SCRAPE RESULT]:\n", result)
        return result if result else "⚠️ No news found."

    except Exception as e:
        print("❌ Scraping failed:", e)
        return f"❌ Scraping error: {e}"

# Defining Agent

In [ ]:
agent = Agent(
    model=model,
    system_prompt=(
        "You are a news agent. If the user wants latest news or a summary "
        "call the scrape() tool and summarize the news. Otherwise respond normally."
    ),
    tools=[scrape]
)

# Chat function for UI

In [ ]:
# -------------------------------------------------------
# CHAT FUNCTION
# -------------------------------------------------------
def chat(user_msg):
  result = agent.run_sync(user_msg)
  return result.output

# User Interface

In [ ]:
# -------------------------------------------------------
# UI
# -------------------------------------------------------
import gradio as gr
with gr.Blocks(css=".gradio-container {max-width: 480px !important; margin: auto;}") as ui:
    gr.Markdown("<h2 style='text-align:center;'>🌍 Real News AI Agent</h2>")

    user_input = gr.Textbox(
        value="Summarize the latest tech news",
        label="Ask anything",
        lines=2
    )

    output = gr.Textbox(label="Result", lines=10)

    manual_button = gr.Button("Run")
    manual_button.click(chat, user_input, output)

ui.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://02122506c911a0bec9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🟦 [TOOL CALLED] scrape() was triggered by the LLM
🔄 Step 1: Fetching news feed...
🔄 Step 2: Extracting top headlines...

✅ [SCRAPE RESULT]:
 - Threads targets podcasters with new features, aiming to become the home for show discussions
  https://techcrunch.com/2025/11/11/threads-targets-podcasters-with-new-features-aiming-to-become-the-place-fans-discuss-shows/

- Immortality startup Eternos pivots to a personal AI that sounds like you
  https://techcrunch.com/2025/11/11/immortality-startup-eternos-pivots-to-a-personal-ai-that-sounds-like-you/

- How AI startups should be thinking about product-market fit
  https://techcrunch.com/2025/11/11/how-ai-startups-should-be-thinking-about-product-market-fit/

- Google Photos adds new AI features for editing, expands AI-powered search to over 100 countries
  https://techcrunch.com/2025/11/11/google-photos-adds-new-ai-features-for-editing-expands-ai-powered-search-to-over-100-countries/

- Wonderful raised $100M Series A to put AI agents on the